In [80]:
import numpy as np

In [81]:
import pandas as pd

In [82]:
params = {}

In [83]:
np.random.seed(10)

In [84]:
params['lsize'] = np.arange(20,50)
np.random.shuffle(params['lsize'])

In [85]:
params['lrate'] = np.linspace(5e-5,5e-3,30)
np.random.shuffle(params['lrate'])

In [86]:
params['bsize'] = np.arange(8,128,4)
np.random.shuffle(params['bsize'])

In [87]:
params['drate'] = np.linspace(0.1,0.6,30)
np.random.shuffle(params['drate'])

In [88]:
params['deg']   = np.linspace(0,29,30)
np.random.shuffle(params['deg'])

In [91]:
df = pd.DataFrame.from_dict(params)

In [93]:
df_sample = df.sample(20)

In [96]:
df_sample.reset_index()

,index,lsize,lrate,bsize,drate,deg
0,3,22,0.002952,72,0.324138,8.0
1,12,38,0.001245,84,0.548276,25.0
2,28,24,0.003293,104,0.531034,29.0
3,22,28,0.000733,40,0.151724,20.0
4,7,47,0.004659,100,0.289655,12.0
5,27,35,0.004317,120,0.255172,16.0
6,16,44,0.002610,52,0.358621,13.0
7,26,20,0.005000,80,0.186207,0.0
8,8,32,0.003634,116,0.272414,21.0
9,23,45,0.000221,16,0.496552,26.0
